# **Inference - Computer Vision for Pneumonia Lung X-Ray Classification**

**`Created by:`**

Michael Nathaniel

---
## **Perkenalan**

**`Informasi Dataset:`**

Dataset ini terdiri dari gambar X-ray dada anak-anak yang dikumpulkan di Guangzhou Women and Children's Medical Center, Guangzhou, China. Dataset ini mencakup total 5.863 gambar X-ray dalam format JPEG, yang dikategorikan menjadi dua kelas: Pneumonia dan Normal. Gambar-gambar ini telah digunakan untuk mengembangkan sistem kecerdasan buatan untuk analisis X-ray dada.

**`Objective:`**

Proyek ini bertujuan mengembangkan model komputer yang dapat melakukan klasifikasi apakah seseorang menderita Pneumonia atau tidak, berdasarkan hasil X-ray paru-paru pasien. Pendekatan yang diterapkan dalam pembuatan model ini adalah konsep Computer Vision, dengan implementasi arsitektur Artificial Neural Network untuk memahami hubungan kompleks antara gambar X-ray dan keberadaan Pneumonia. Prinsip dasar proyek ini melibatkan pelatihan model untuk mengenali pola dan fitur pada gambar X-ray yang menunjukkan adanya Pneumonia. Dengan demikian, proyek ini bersifat esensial dalam pengaplikasian teknologi kecerdasan buatan dalam bidang diagnostik medis.

**`Problems:`** 

Buatlah model komputer yang dapat melakukan klasifikasi apakah seseorang menderita Pneumonia atau tidak, berdasarkan hasil X-ray paru-paru pasien.

---
## **Model Inference**

### **1. Import Modul**

Kita akan mengimport modul modul yang diperlukan terlebih dahulu

In [1]:
import requests
import numpy as np

from PIL import Image
from io import BytesIO
from keras.models import load_model
from keras.preprocessing.image import img_to_array

### **2. Load Model**

Pertama-tama, kita akan meload model terlebih dahulu.

In [2]:
# Melakukan import model
model = load_model("model_final.h5")

### **3. Input Data Inference**

In [3]:
# Membuat URL data inferece
xray_paru_normal_image_url = 'https://prod-images-static.radiopaedia.org/images/220869/76052f7902246ff862f52f5d3cd9cd_big_gallery.jpg'
xray_paru_pneumonia_image_url = 'https://img.medscapestatic.com/pi/meds/ckb/58/16958tn.jpg'

### **4. Jalankan Inference**

In [4]:
# Buat fungsi untuk preprocessing foto
def preprocess_image(image_path):
    # Download foto dari URL
    response = requests.get(image_path)
    img = Image.open(BytesIO(response.content))
    
    # Resize foto menjadi sesuai dengan target size
    img = img.resize((150, 150))
    
    # Convert foto menjadi numpy array
    img_array = img_to_array(img)
    
    # Mengekspnasi dimensi and normalisasi pixel values
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    return img_array

In [5]:
# Membuat fungsi untuk melakukan prediksi
def predict_pneumonia(image_path):
    # melakukan preproces foto
    img_array = preprocess_image(image_path)
    
    # Menjalankan prediksi
    prediction = model.predict(img_array)
    
    # mengkonversi hasil prediksi menjadi dapat dibaca manusia
    if prediction[0][0] > 0.7:
        return "Pneumonia"
    else:
        return "Normal"

In [6]:
normal_prediction = predict_pneumonia(xray_paru_normal_image_url)
pneumonia_prediction = predict_pneumonia(xray_paru_pneumonia_image_url)

1/1 [==============================] - 0s 49ms/step


In [7]:
# Display the predictions
print(f"Prediksi untuk foto paru-paru normal: {normal_prediction}")
print(f"Prediksi untuk foto pari-paru pneumonia: {pneumonia_prediction}")

Prediksi untuk foto paru-paru normal: Normal
Prediksi untuk foto pari-paru pneumonia: Pneumonia
